### Loading dataset

In [1]:
from google.colab import drive
from google.colab import files
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to dataset file on Google Drive
folder_path2014 = "/content/drive/MyDrive/SEM 06/6SP/Raw Operational Data from Enterprise Application - KAGGLE.zip (Unzipped Files)/2014"
folder_path2015 = "/content/drive/MyDrive/SEM 06/6SP/Raw Operational Data from Enterprise Application - KAGGLE.zip (Unzipped Files)/2015"

folders = [folder_path2014, folder_path2015]

# Log file list
log_files = []

file_path = "/content/drive/MyDrive/SEM 06/6SP/all_log_files.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
  for folder in folders:
    for dirname, _, filenames in os.walk(folder):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            # Write content to the file
            file.write(os.path.join(dirname, filename))
            file.write("\n")


print("All file names saved to all_log_files.txt ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All file names saved to all_log_files.txt 


In [2]:
import random

# Set the range for random numbers
start_range = 1
end_range = 18337
num_samples = 100

# Generate a list of random numbers
random_numbers = random.sample(range(start_range, end_range + 1), num_samples)

# Print the list of random numbers
print(random_numbers)

[2717, 1928, 15998, 13102, 2475, 2946, 3202, 4221, 13337, 5960, 2197, 12459, 18289, 7205, 14482, 10382, 13329, 6120, 9511, 2186, 1162, 12022, 12036, 16081, 11029, 5559, 4796, 53, 11824, 10998, 15258, 356, 1049, 8595, 95, 4250, 7062, 7283, 16745, 1671, 9164, 4433, 8433, 9181, 8866, 15426, 11103, 10817, 16854, 5513, 1987, 9156, 18282, 9379, 17820, 4223, 12138, 17733, 10276, 14700, 4157, 6887, 7877, 5014, 3548, 4992, 995, 16648, 6504, 6696, 5172, 14665, 4768, 1110, 1554, 6226, 7487, 6351, 10576, 16446, 13158, 14736, 5578, 4003, 5642, 15246, 5124, 7068, 12894, 17160, 8336, 12186, 12731, 11026, 7128, 443, 15133, 14372, 16147, 9335]


### Creating dataframes from log files


In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# List of dataframes
dfs = []


# file_path = "/content/drive/MyDrive/SEM 06/6SP/all_log_files.txt"

i = 0 # For line number

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the file line by line
    for line in file:
      i += 1
      if i > max(random_numbers):
        break
      if i in random_numbers:
        dfs.append( pd.read_csv(line.strip(),low_memory=False))


print(len(dfs))


100


### Viewing some df from dfs

In [4]:
dfs[2].head()

,Date,\User\users,\Memory\total,\Uptime\up,\Memory\buffers,\Swap\used,\Swap\free,\Swap\total,\Tasks\zombie,\Tasks\total,...,\Cpu\si,\Tasks\sleeping,\Cpu\hi,\Load\avg,\Memory\used,\Cpu\us,\Cpu\sy,\Cpu\ni,\Cpu\st,Unnamed: 27
0,11.08.2014 07:35:46.154,-1,-1,-1,-1,"28,494,004,224","4,978,638,848","33,473,691,648",-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN
1,11.08.2014 07:36:49.348,-1,-1,-1,-1,"28,490,858,496","4,981,784,576","33,473,691,648",-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN
2,11.08.2014 07:37:56.314,-1,-1,-1,-1,"28,549,578,752","4,923,064,320","33,473,691,648",-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN
3,11.08.2014 07:38:58.021,-1,-1,-1,-1,"28,494,004,224","4,979,687,424","33,473,691,648",-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN
4,11.08.2014 07:39:58.845,-1,-1,-1,-1,"28,651,290,624","4,822,401,024","33,473,691,648",-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN


### Creating log file clusters based on column names

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


# # List of log file paths
# log_files = ['log_file1.csv', 'log_file2.csv', 'log_file3.csv', ...]

# Load log files into separate DataFrames
dataframes = dfs
# for file in log_files:
#     df = pd.read_csv(file)
#     dataframes.append(df)

# Extract column names from each DataFrame
column_names = []
for df in dataframes:
    column_names.append(df.columns.tolist())

# print(column_names)



In [6]:
file_path = "/content/drive/MyDrive/SEM 06/6SP/columns.txt"
# Open the file in write mode
with open(file_path, 'w') as file:
  for cols in column_names:
    for col in cols:
      file.write(col)
      file.write('\n')

with open(file_path, 'r') as file:
    lines = file.readlines()


# Create feature matrix
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(lines)

# Apply dimensionality reduction
pca = PCA(n_components=2)  # You can adjust the number of components
X_reduced = pca.fit_transform(X.toarray())

# Perform clustering with DBSCAN
scaler = StandardScaler()
X_reduced_scaled = scaler.fit_transform(X_reduced)

# dbscan = DBSCAN(eps=0.5, min_samples=2)  # Adjust the eps and min_samples values <----------------------------------
# clusters = dbscan.fit_predict(X_reduced_scaled)


# # Step 8: Assign log files to clusters
# log_files_clusters = {}
# for i, file in enumerate(log_files):
#     log_files_clusters[file] = clusters[i]

# # Print the log file and its corresponding cluster
# for file, cluster in log_files_clusters.items():
#     print(f"Log file: {file}, Cluster: {cluster}")

In [12]:
dbscan = DBSCAN(eps=0.5, min_samples=2)  # Adjust the eps and min_samples values <----------------------------------
clusters = dbscan.fit_predict(X_reduced_scaled)

print(clusters)


# Assign log files to clusters
log_files_clusters = {}
for i, file in enumerate(log_files):
    log_files_clusters[file] = clusters[i]



[0 0 0 ... 0 0 0]
Done!
